# I need to duplicate the fragility table a bunch of times for the training 

# File Description

I need to check the combined fragility. Under certain circumstances, it doesn't give sensible results (e.g. Shaking DS1, Tsunami DS2, Landslide DS0, combined is all 20% for some reason???

# File Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import itertools
import os
import json
from datetime import datetime
import pysmile
import pysmile_license
import sys
import json
sys.path.append('/src/python_classes')
import rpy2
# os.environ['R_HOME'] = 'C:\ProgramData\Anaconda3\Lib\R'
# %load_ext rpy2.ipython
!jupyter nbextension enable --py --sys-prefix ipyleaflet
from ipywidgets import interact, interactive, fixed, interact_manual
# from ipyleaflet import *
# import ipywidgets as widgets
from matplotlib.animation import FuncAnimation
# import geojson
import folium
from colormap import rgb2hex
# import rpy2
# os.environ['R_HOME'] = '/lib/R'
# %load_ext rpy2.ipython
from folium.plugins import FloatImage
from collections import ChainMap
import re

# from BNModel import BNModel


Traceback (most recent call last):
  File "C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\Scripts\jupyter-nbextension-script.py", line 9, in <module>
    sys.exit(main())
  File "C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\lib\site-packages\jupyter_core\application.py", line 269, in launch_instance
    return super().launch_instance(argv=argv, **kwargs)
  File "C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
    app.start()
  File "C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\lib\site-packages\notebook\nbextensions.py", line 972, in start
    super().start()
  File "C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\lib\site-packages\jupyter_core\application.py", line 258, in start
    self.subapp.start()
  File "C:\Users\shannonb\AppData\Local\miniforge3\envs\pac_model\lib\site-packages\notebook\nbextensions.py", line 882, in start
    self.toggle_nbe

ModuleNotFoundError: No module named 'BNModel'

In [ ]:
net = pysmile.Network()

# Load Hazard Bins

In [ ]:
with open("bin_definitions.json") as f:
    bin_defs_dict = json.load(f)

In [ ]:
bin_defs_dict

# Methods for combining DS

In [ ]:
damage_states_array = np.array((0,1,2,3,4))
landslide_states_array = np.array((0,4))

fragility_array = np.array(('Shaking','Tsunami','Landslide'))

In [ ]:
rows = list(itertools.product(list(itertools.product(damage_states_array,damage_states_array)),landslide_states_array))

df_fragilities = pd.DataFrame.from_dict(dict(ChainMap(*[{i:{
    'Shaking':x[0][0],
    'Tsunami':x[0][1],
    'Landslide':x[1]}} for i,x in zip(np.arange(0,len(rows),1),rows)]))).T

#### But we can put in percentages.. it doesn't have to be 100% into the one state ####

# Rules:
# Pick the max damage state. Start with this as the combined damage state.
# If there are multiple hazards of the same damage state, add one to the combined damage state
# Cap the damage state at 3

for index,row in df_fragilities.iterrows():
    comb_frag = np.max(row)
    # See how many hazards have the max damage state:
    number_with_max_damage = list(row).count(comb_frag)
#     # If there are more than 2, add one to the combined damage state (unless your at max or no damage)
#     if (number_with_max_damage>=2)&(comb_frag!=3)&(comb_frag!=0):
#         comb_frag+=1
    
    # Add to the fragility
    df_fragilities.loc[index,'Combined'] = int(comb_frag)
    
# Rename the columns to include fragility
[df_fragilities.rename(columns={x:x+'Fragility'},inplace=True) for x in df_fragilities if x!='Combined']
    

In [ ]:
# Replace the integers with strings 
df_fragilities = df_fragilities.replace(0,'DS0')
df_fragilities = df_fragilities.replace(1,'DS1')
df_fragilities = df_fragilities.replace(2,'DS2')
df_fragilities = df_fragilities.replace(3,'DS3')
df_fragilities = df_fragilities.replace(4,'DS4')


In [ ]:
df_fragilities.reset_index(drop=True,inplace=True)
df_fragilities

In [ ]:
df_fragilities_melt = df_fragilities.melt(['ShakingFragility','TsunamiFragility','LandslideFragility'])
df_fragilities_melt = df_fragilities_melt.sort_values(['ShakingFragility','TsunamiFragility','LandslideFragility'])
df_fragilities_melt

In [ ]:
net.add_node(pysmile.NodeType.CPT,'CombinedFragility')
for outcome in ['DS0','DS1','DS2','DS3','DS4']:
    net.add_outcome('CombinedFragility',outcome)

net.add_node(pysmile.NodeType.CPT,'ShakingFragility')
for outcome in ['DS0','DS1','DS2','DS3','DS4']:
    net.add_outcome('ShakingFragility',outcome)
net.add_arc('ShakingFragility','CombinedFragility')

net.add_node(pysmile.NodeType.CPT,'TsunamiFragility')
for outcome in ['DS0','DS1','DS2','DS3','DS4']:
    net.add_outcome('TsunamiFragility',outcome)
net.add_arc('TsunamiFragility','CombinedFragility')

net.add_node(pysmile.NodeType.CPT,'LandslideFragility')
for outcome in ['DS0','DS4']:
    net.add_outcome('LandslideFragility',outcome)
net.add_arc('LandslideFragility','CombinedFragility')


for node in ['CombinedFragility','ShakingFragility','TsunamiFragility','LandslideFragility']:
    net.delete_outcome(node,'State0')
    net.delete_outcome(node,'State1')

In [ ]:
len(df_fragilities_melt)*5

In [ ]:
cpt_list = []
for index,row in df_fragilities_melt.iterrows():
    if row.value=='DS0':
        lst = [1,0,0,0,0]
    elif row.value=='DS1':
        lst = [0,1,0,0,0]
    elif row.value=='DS2':
        lst = [0,0,1,0,0]
    elif row.value=='DS3':
        lst = [0,0,0,1,0]
    elif row.value=='DS4':
        lst = [0,0,0,0,1]
        
    cpt_list = cpt_list+lst
    
net.set_node_definition('CombinedFragility',cpt_list)

In [ ]:
df_fragilities[(df_fragilities.TsunamiFragility=='DS2')&(df_fragilities.LandslideFragility=='DS0')&(df_fragilities.ShakingFragility=='DS1')]


In [ ]:
pga_general = np.arange(0,5,1)
PGA1 = pga_general
PGA2 = pga_general
PGA3 = pga_general
LSA = np.arange(0,3,1)

rows = list(itertools.product(list(itertools.product(list(itertools.product(PGA1,PGA2)),PGA3)),LSA))

df_pga_cpt = pd.DataFrame.from_dict(dict(ChainMap(*[{i:{
    'ShakingFragilityA':x[0][0][0],
    'ShakingFragilityB':x[0][0][1],
    'ShakingFragilityC':x[0][1],
    'LiquefactionFragility':x[1]}} for i,x in zip(np.arange(0,len(rows),1),rows)]))).T
    
df_pga_cpt.loc[df_pga_cpt.LiquefactionFragility==0,'ShakingFragility'] = df_pga_cpt.loc[df_pga_cpt.LiquefactionFragility==0,'ShakingFragilityA']
df_pga_cpt.loc[df_pga_cpt.LiquefactionFragility==1,'ShakingFragility'] = df_pga_cpt.loc[df_pga_cpt.LiquefactionFragility==1,'ShakingFragilityB']
df_pga_cpt.loc[df_pga_cpt.LiquefactionFragility==2,'ShakingFragility'] = df_pga_cpt.loc[df_pga_cpt.LiquefactionFragility==2,'ShakingFragilityC']

# df_pga_cpt.loc[df_pga_cpt.PGAt==0]
# df_pga_cpt.loc[df_pga_cpt.PGAt==1]
# df_pga_cpt.loc[df_pga_cpt.PGAt==2]
# df_pga_cpt.loc[df_pga_cpt.PGAt==3]
# df_pga_cpt.loc[df_pga_cpt.PGAt==4]

df_pga_cpt = df_pga_cpt.replace(0,'DS0')
df_pga_cpt = df_pga_cpt.replace(1,'DS1')
df_pga_cpt = df_pga_cpt.replace(2,'DS2')
df_pga_cpt = df_pga_cpt.replace(3,'DS3')
df_pga_cpt = df_pga_cpt.replace(4,'DS4')

df_pga_cpt

In [ ]:
# for index,row in df_fragilities.iterrows():
#     df = df_pga_cpt[df_pga_cpt.ShakingFragility==row.ShakingFragility]
#     df2 = pd.DataFrame([row]*len(df)).merge(df,left_on='ShakingFragility',right_on='ShakingFragility')
#     if index!=0:
#         df3 = df3.append(df2)
#     else:
#         df3 = df2.copy()

In [ ]:
# df_fragilities = df3

In [ ]:
# df_fragilities

In [ ]:
# df_fragilities[['ShakingFragilityA','ShakingFragilityB','ShakingFragilityC','LiquefactionFragility','ShakingFragility']].loc[100]

# Create a BN using this table

In [ ]:
# frag_data_dict = {k:np.array(df_fragilities[k]) for k in list(df_fragilities)}

In [ ]:
# frag_data_dict

In [ ]:
# frag_file_label = 'fragility'

# #### Don't include spaces in bin names. if no discretisation, just leave out that key
# frag_model_dict = {
#     'variables':{
#         'ShakingFragility':{
#             'label':'ShakingFragility',
#             'child_nodes':['Combined'],
#             'bins':['DS0','DS1','DS2','DS3','DS4']
#         },
#         'TsunamiFragility':{
#             'label':'TsunamiFragility',
#             'child_nodes':['Combined'],
#             'bins':['DS0','DS1','DS2','DS3','DS4']
#         },
#         'LandslideFragility':{
#             'label':'LandslideFragility',
#             'child_nodes':['Combined'],
#             'bins':['DS0','DS4']
#         },
#         'Combined':{
#             'label':'Combined',
#             'child_nodes':[],
#             'bins':['DS0','DS1','DS2','DS3','DS4']
#         }
#     },
#     'training_frac':0.8,
#     'bootstrap_reps':1
# }

In [ ]:
# frag_model_dict = BNModel().bootstrap_data(frag_model_dict,frag_data_dict,df_fragilities)

In [ ]:
# frag_model_dict = BNModel().discretiser(frag_model_dict,[])

In [ ]:
# BNModel().save_dataset(frag_model_dict,frag_file_label)

In [ ]:
# frag_model_dict = BNModel().create_SM(frag_model_dict,frag_file_label)

In [ ]:
# Set evidence and get beliefs
# frag_model_dict = BNModel().update_evidence(frag_model_dict)

# Create Additional Shaking Nodes

In [ ]:
# frag_model_dict_copy = frag_model_dict.copy()
# net = frag_model_dict_copy['model'][0]['net']

In [ ]:
net.add_node(pysmile.NodeType.CPT,'ShakingFragilityA')
for outcome in ['DS0','DS1','DS2','DS3','DS4']:
    net.add_outcome('ShakingFragilityA',outcome)
net.add_arc('ShakingFragilityA','ShakingFragility')

net.add_node(pysmile.NodeType.CPT,'ShakingFragilityB')
for outcome in ['DS0','DS1','DS2','DS3','DS4']:
    net.add_outcome('ShakingFragilityB',outcome)
net.add_arc('ShakingFragilityB','ShakingFragility')

net.add_node(pysmile.NodeType.CPT,'ShakingFragilityC')
for outcome in ['DS0','DS1','DS2','DS3','DS4']:
    net.add_outcome('ShakingFragilityC',outcome)
net.add_arc('ShakingFragilityC','ShakingFragility')

net.add_node(pysmile.NodeType.CPT,'LiquefactionFragility')
for outcome in ['DS0','DS1','DS2']:
    net.add_outcome('LiquefactionFragility',outcome)
net.add_arc('LiquefactionFragility','ShakingFragility')

for node in ['ShakingFragilityA','ShakingFragilityB','ShakingFragilityC','LiquefactionFragility']:
    net.delete_outcome(node,'State0')
    net.delete_outcome(node,'State1')

In [ ]:
df_pga_cpt.set_index('ShakingFragility')#.pivot(columns=['ShakingFragilityA','ShakingFragilityB','ShakingFragilityC','LiquefactionFragility'])


In [ ]:
df_pga_cpt_melt = df_pga_cpt.melt(['ShakingFragilityA','ShakingFragilityB','ShakingFragilityC','LiquefactionFragility'])
df_pga_cpt_melt = df_pga_cpt_melt.sort_values(['ShakingFragilityA','ShakingFragilityB','ShakingFragilityC','LiquefactionFragility'])

cpt_list = []
for index,row in df_pga_cpt_melt.iterrows():
    if row.value=='DS0':
        lst = [1,0,0,0,0]
    elif row.value=='DS1':
        lst = [0,1,0,0,0]
    elif row.value=='DS2':
        lst = [0,0,1,0,0]
    elif row.value=='DS3':
        lst = [0,0,0,1,0]
    elif row.value=='DS4':
        lst = [0,0,0,0,1]
        
    cpt_list = cpt_list+lst
    
net.set_node_definition('ShakingFragility',cpt_list)


In [ ]:
for node in ['ShakingFragilityA','ShakingFragilityB','ShakingFragilityC']:
    net.set_node_definition(node,[1/5]*5)
net.set_node_definition('LiquefactionFragility',[1/3]*3)

In [ ]:

# # Delete the premade states
# net.delete_outcome(new_node_name,'State0')
# net.delete_outcome(new_node_name,'State1')
# # Create connection between the hazard and fragility functions

# net.add_arc(new_node_name,child_node_name)
# print(new_node_name)
# print(child_node_name)
# # Set the values of the conditional probability table for the fragility node
# if new_node_name=='Shaking':
#     cpt_value = \
#         list(df_ds_shaking[df_ds_shaking.index==earthquake_low].T[earthquake_low])+\
#         list(df_ds_shaking[df_ds_shaking.index==earthquake_medium].T[earthquake_medium])+\
#         list(df_ds_shaking[df_ds_shaking.index==earthquake_high].T[earthquake_high])
# elif new_node_name=='Landslide':
#     cpt_value = \
#         list(df_ds_landslide[df_ds_landslide.index==landslide_no].T[landslide_no])+\
#         list(df_ds_landslide[df_ds_landslide.index==landslide_yes].T[landslide_yes])
# elif new_node_name=='Tsunami':
#     cpt_value = \
#         list(df_ds_tsunami[df_ds_tsunami.index==tsunami_zero].T[tsunami_zero])+\
#         list(df_ds_tsunami[df_ds_tsunami.index==tsunami_low].T[tsunami_low])+\
#         list(df_ds_tsunami[df_ds_tsunami.index==tsunami_medium].T[tsunami_medium])+\
#         list(df_ds_tsunami[df_ds_tsunami.index==tsunami_high].T[tsunami_high])
# elif new_node_name=='Liquefaction':
#     cpt_value = \
#         list(df_ds_liquefaction[df_ds_liquefaction.index==liquefaction_zero].T[liquefaction_zero])+\
#         list(df_ds_liquefaction[df_ds_liquefaction.index==liquefaction_low].T[liquefaction_low])+\
#         list(df_ds_liquefaction[df_ds_liquefaction.index==liquefaction_medium].T[liquefaction_medium])+\
#         list(df_ds_liquefaction[df_ds_liquefaction.index==liquefaction_high].T[liquefaction_high])
# net.set_node_definition(child_node_name,cpt_value)
# net.set_node_definition(new_node_name,[1/len(outcomes)]*len(outcomes))


# Creating fragility functions

## Earthquakes

In [ ]:
earthquake_bins = {
    'low':0.5001, #0-0.25
    'medium':0.7001, #0.25-0.75
    'high':4.0001 #>.75
}

In [ ]:
ax = plt.subplot2grid((1,1),(0,0))

# Get max mean pga
mu_ds1 = 0.0588#bins['minimum']
mu_ds2 = 0.0952#2*earthquake_diff+bins['minimum']
mu_ds3 = 0.30808#1*earthquake_diff+bins['minimum']
mu_ds4 = 0.413616#4*earthquake_diff+bins['minimum']

sigma_ds1 = 0.54#0.00000001
sigma_ds2 = 0.58#1
sigma_ds3 = 0.49#1
sigma_ds4 = 0.57#1

IM = IM = np.arange(0.0001,10,0.1)#np.arange(bins['minimum'],earthquake_max,(earthquake_max-bins['minimum'])/1000)

from scipy.stats import lognorm
import math

# P_ds1 = [lognorm(s=sigma_ds1).cdf(im)*mu_ds1/lognorm(s=sigma_ds1).mean() for im in IM]
# P_ds2 = [lognorm(s=sigma_ds2).cdf(im)*mu_ds2/lognorm(s=sigma_ds2).mean() for im in IM]
# P_ds3 = [lognorm(s=sigma_ds3).cdf(im)*mu_ds3/lognorm(s=sigma_ds3).mean() for im in IM]
# P_ds4 = [lognorm(s=sigma_ds4).cdf(im)*mu_ds4/lognorm(s=sigma_ds4).mean() for im in IM]

P_ds1a = [lognorm(s=sigma_ds1,scale=math.exp(mu_ds1*0.98)).cdf(im) for im in IM]
P_ds2a = [lognorm(s=sigma_ds2,scale=math.exp(mu_ds2*0.98)).cdf(im) for im in IM]
P_ds3a = [lognorm(s=sigma_ds3,scale=math.exp(mu_ds3*0.98)).cdf(im) for im in IM]
P_ds4a = [lognorm(s=sigma_ds4,scale=math.exp(mu_ds4*0.98)).cdf(im) for im in IM]

P_ds1b = [lognorm(s=sigma_ds1,scale=math.exp(mu_ds1*.9)).cdf(im) for im in IM]
P_ds2b = [lognorm(s=sigma_ds2,scale=math.exp(mu_ds2*.9)).cdf(im) for im in IM]
P_ds3b = [lognorm(s=sigma_ds3,scale=math.exp(mu_ds3*.9)).cdf(im) for im in IM]
P_ds4b = [lognorm(s=sigma_ds4,scale=math.exp(mu_ds4*.9)).cdf(im) for im in IM]

P_ds1c = [lognorm(s=sigma_ds1,scale=math.exp(mu_ds1*.8)).cdf(im) for im in IM]
P_ds2c = [lognorm(s=sigma_ds2,scale=math.exp(mu_ds2*.8)).cdf(im) for im in IM]
P_ds3c = [lognorm(s=sigma_ds3,scale=math.exp(mu_ds3*.8)).cdf(im) for im in IM]
P_ds4c = [lognorm(s=sigma_ds4,scale=math.exp(mu_ds4*.8)).cdf(im) for im in IM]


P_ds1d = [lognorm(s=sigma_ds1,scale=math.exp(mu_ds1*.6)).cdf(im) for im in IM]
P_ds2d = [lognorm(s=sigma_ds2,scale=math.exp(mu_ds2*.6)).cdf(im) for im in IM]
P_ds3d = [lognorm(s=sigma_ds3,scale=math.exp(mu_ds3*.6)).cdf(im) for im in IM]
P_ds4d = [lognorm(s=sigma_ds4,scale=math.exp(mu_ds4*.6)).cdf(im) for im in IM]


l1 = plt.plot(IM,P_ds1a,c='r')
l2 = plt.plot(IM,P_ds2a,c='b')
l3 = plt.plot(IM,P_ds3a,c='k')
l4 = plt.plot(IM,P_ds4a,c='g')

l5 = plt.plot(IM,P_ds1b,ls='--',c='r')
l6 = plt.plot(IM,P_ds2b,ls='--',c='b')
l7 = plt.plot(IM,P_ds3b,ls='--',c='k')
l8 = plt.plot(IM,P_ds4b,ls='--',c='g')

l9 = plt.plot(IM,P_ds1c,ls='-.',c='r')
l10 = plt.plot(IM,P_ds2c,ls='-.',c='b')
l11 = plt.plot(IM,P_ds3c,ls='-.',c='k')
l12 = plt.plot(IM,P_ds4c,ls='-.',c='g')

# ax.plot([bins['low'],bins['low']],[0,1],ls='--',c='k')
# ax.plot([bins['medium'],bins['medium']],[0,1],ls='--',c='k')
# ax.plot([bins['high'],bins['high']],[0,1],ls='--',c='k')

ax.legend(['DS1','DS2','DS3','DS4'])

# plt.plot(IM,P_ds0)
# plt.plot(IM,P_ds1)
# plt.plot(IM,P_ds2)
# plt.plot(IM,P_ds3)


In [ ]:
# # mu_ds1 = 0.25
# # mu_ds2 = 0.55
# # mu_ds3 = 1.28
# # mu_ds4 = 2.01

# # # Shaking
# # mu_list = [0.25,0.55,1.25,2.01]
# # sigma_list = [0.64,0.64,0.64,0.64]
# ds_list = ['DS0','DS1','DS2','DS3','DS4']
# IM = np.arange(0.0001,10,0.1)
# factors = [1,0.75,0.55]

# # Tsunami
# # Get max mean pga
# mu_list = [0.178,0.4255,1.05,1.5]
# sigma_list = [1.1462,0.9395,0.7155,0.5288]
# factor = 1
# IM = np.arange(0.0001,3,0.1)


# transform_fragility_funcs(factor,mu_list,sigma_list,ds_list,IM)

# pd.DataFrame.from_dict({ds:[lognorm(s=sigma,scale=math.exp(mu*factor)).cdf(im) for im in IM] for mu,sigma,ds in zip(mu_list,sigma_list,ds_list)})



In [ ]:
df_ds_shaking_a = pd.DataFrame.from_dict({"DS1":P_ds1a,"DS2":P_ds2a,"DS3":P_ds3a,"DS4":P_ds4a})
df_ds_shaking_a.index = IM

df_ds_shaking_b = pd.DataFrame.from_dict({"DS1":P_ds1b,"DS2":P_ds2b,"DS3":P_ds3b,"DS4":P_ds4b})
df_ds_shaking_b.index = IM

df_ds_shaking_c = pd.DataFrame.from_dict({"DS1":P_ds1c,"DS2":P_ds2c,"DS3":P_ds3c,"DS4":P_ds4c})
df_ds_shaking_c.index = IM

df_ds_shaking_d = pd.DataFrame.from_dict({"DS1":P_ds1d,"DS2":P_ds2d,"DS3":P_ds3d,"DS4":P_ds4d})
df_ds_shaking_d.index = IM

In [ ]:
def normalise(df_ds_shaking):
    df_ds_shaking["DS3"] = df_ds_shaking.DS3-df_ds_shaking.DS4
    df_ds_shaking["DS2"] = df_ds_shaking.DS2-df_ds_shaking.DS3-df_ds_shaking.DS4
    df_ds_shaking["DS1"] = df_ds_shaking.DS1-df_ds_shaking.DS2-df_ds_shaking.DS3-df_ds_shaking.DS4

    # Now add zero
    ds_shaking_dict = {}

    for index,row in df_ds_shaking.iterrows():
        row['DS0'] = (1-np.sum(row))
        ds_shaking_dict.update({
            index:row
        })

    df_ds_shaking = pd.DataFrame.from_dict(ds_shaking_dict,orient='index')
    
    return(df_ds_shaking)

In [ ]:
df_ds_shaking_a = normalise(df_ds_shaking_a)
df_ds_shaking_b = normalise(df_ds_shaking_b)
df_ds_shaking_c = normalise(df_ds_shaking_c)
df_ds_shaking_d = normalise(df_ds_shaking_d)

In [ ]:
l0 = plt.plot(df_ds_shaking_a.DS0)
l1 = plt.plot(df_ds_shaking_a.DS1)
l2 = plt.plot(df_ds_shaking_a.DS2)
l3 = plt.plot(df_ds_shaking_a.DS3)
l4 = plt.plot(df_ds_shaking_a.DS4)

plt.xlim([0,5])

plt.legend(['DS0','DS1','DS2','DS3','DS4'])
plt.title('Shaking')
plt.xlabel('PGA')
plt.ylabel('Prob')

In [ ]:
df_ds_shaking_a = df_ds_shaking_a[['DS0','DS1','DS2','DS3','DS4']]
df_ds_shaking_b = df_ds_shaking_b[['DS0','DS1','DS2','DS3','DS4']]
df_ds_shaking_c = df_ds_shaking_c[['DS0','DS1','DS2','DS3','DS4']]
df_ds_shaking_d = df_ds_shaking_d[['DS0','DS1','DS2','DS3','DS4']]


## Tsunami

In [ ]:
tsunami_bins = {
    'zero':0.0001, #0
    'low':0.2001, #<0.2
    'medium':1.5001, #0.2-1
    'high':2.7001 #>1
}

In [ ]:
ax = plt.subplot2grid((1,1),(0,0))

# Get max mean pga
mu_ds1 = 0.5#bins['minimum']
mu_ds2 = 1#2*earthquake_diff+bins['minimum']
mu_ds3 = 2#3*earthquake_diff+bins['minimum']
mu_ds4 = 3.5#4*earthquake_diff+bins['minimum']

sigma_ds1 = 0.54#0.00000001
sigma_ds2 = 0.58#1
sigma_ds3 = 0.49#1
sigma_ds4 = 0.57#1

IM = IM = np.arange(0.0001,3,0.1)#np.arange(bins['minimum'],earthquake_max,(earthquake_max-bins['minimum'])/1000)

from scipy.stats import lognorm
import math

P_ds1 = [lognorm(s=sigma_ds1,scale=math.exp(mu_ds1)).cdf(im) for im in IM]
P_ds2 = [lognorm(s=sigma_ds2,scale=math.exp(mu_ds2)).cdf(im) for im in IM]
P_ds3 = [lognorm(s=sigma_ds3,scale=math.exp(mu_ds3)).cdf(im) for im in IM]
P_ds4 = [lognorm(s=sigma_ds4,scale=math.exp(mu_ds4)).cdf(im) for im in IM]


l1 = plt.plot(IM,P_ds1)
l2 = plt.plot(IM,P_ds2)
l3 = plt.plot(IM,P_ds3)
l4 = plt.plot(IM,P_ds4)

# ax.plot([bins['low'],bins['low']],[0,1],ls='--',c='k')
# ax.plot([bins['medium'],bins['medium']],[0,1],ls='--',c='k')
# ax.plot([bins['high'],bins['high']],[0,1],ls='--',c='k')

ax.legend(['DS1','DS2','DS3','DS4'])

# plt.plot(IM,P_ds0)
# plt.plot(IM,P_ds1)
# plt.plot(IM,P_ds2)
# plt.plot(IM,P_ds3)


# Add a rule to stop the crossover

In [ ]:
df_ds_tsunami = pd.DataFrame.from_dict({"DS1":P_ds1,"DS2":P_ds2,"DS3":P_ds3,"DS4":P_ds4})
df_ds_tsunami.index = IM

In [ ]:
df_ds_tsunami["DS3"] = df_ds_tsunami.DS3-df_ds_tsunami.DS4
df_ds_tsunami["DS2"] = df_ds_tsunami.DS2-df_ds_tsunami.DS3-df_ds_tsunami.DS4
df_ds_tsunami["DS1"] = df_ds_tsunami.DS1-df_ds_tsunami.DS2-df_ds_tsunami.DS3-df_ds_tsunami.DS4

# Now add zero
ds_tsunami_dict = {}

for index,row in df_ds_tsunami.iterrows():
    row['DS0'] = (1-np.sum(row))
    ds_tsunami_dict.update({
        index:row
    })
    
df_ds_tsunami = pd.DataFrame.from_dict(ds_tsunami_dict,orient='index')


In [ ]:
l0 = plt.plot(df_ds_tsunami.DS0)
l1 = plt.plot(df_ds_tsunami.DS1)
l2 = plt.plot(df_ds_tsunami.DS2)
l3 = plt.plot(df_ds_tsunami.DS3)

plt.legend(['DS0','DS1','DS2','DS3'])
plt.title('Tsunami')
plt.xlabel('Height (m)')
plt.ylabel('Prob')

In [ ]:
df_ds_tsunami = df_ds_tsunami[['DS0','DS1','DS2','DS3','DS4']]
df_ds_tsunami

## Liquefaction

In [ ]:
liquefaction_bins = {
    'A':0, #<0.2
    'B':1, #0.2-1
    'C':2,
    'D':3#>1
}


## Landslide

In [ ]:
# It's binary, so just make up two rows
ds_landslide_dict = {
    '0':{
        'DS0':1.0,
        'DS4':0.0
    },
    '1':{
        'DS0':0.0,
        'DS4':1.0
    }
}

df_ds_landslide = pd.DataFrame.from_dict(ds_landslide_dict,orient='index')

In [ ]:
df_ds_landslide = pd.DataFrame.from_dict(ds_landslide_dict,orient='index')

## Find the prior distribution for this hazard

## Define Low, Medium and High

In [ ]:
# ## Make up the definition of low, medium and high
# len_pga = len(list(df_ds_shaking.index))
# pga_low = list(df_ds_shaking.index)[int((len_pga-1)/4)]
# pga_medium = list(df_ds_shaking.index)[int(2*(len_pga-1)/4)]
# pga_high = list(df_ds_shaking.index)[int(3*(len_pga-1)/4)]


In [ ]:
earthquake_bins['medium']

In [ ]:
min(list(df_ds_shaking_b.index), key=lambda x:abs(x-earthquake_bins['medium']))

In [ ]:
earthquake_low = min(list(df_ds_shaking_a.index), key=lambda x:abs(x-earthquake_bins['low']))
earthquake_medium = min(list(df_ds_shaking_a.index), key=lambda x:abs(x-earthquake_bins['medium']))
earthquake_high = min(list(df_ds_shaking_a.index), key=lambda x:abs(x-earthquake_bins['high']))

liquefaction_0 = liquefaction_bins['A']
liquefaction_1 = liquefaction_bins['B']
liquefaction_2 = liquefaction_bins['C']
liquefaction_3 = liquefaction_bins['D']

tsunami_zero = min(list(df_ds_tsunami.index), key=lambda x:abs(x-tsunami_bins['zero']))
tsunami_low = min(list(df_ds_tsunami.index), key=lambda x:abs(x-tsunami_bins['low']))
tsunami_medium = min(list(df_ds_tsunami.index), key=lambda x:abs(x-tsunami_bins['medium']))
tsunami_high = min(list(df_ds_tsunami.index), key=lambda x:abs(x-tsunami_bins['high']))

landslide_no = '0'
landslide_yes = '1'


# Set Conditional Prob Tables

In [ ]:
net.write_file("BN_simple.xdsl")

In [ ]:
list_of_nodes = net.get_all_nodes()
# # list_of_nodes.pop(net.get_node('Combined'))
# # list_of_nodes.pop(net.get_node('ShakingFragility'))
# # list_of_nodes.pop(net.get_node('ShakingFragilityA'))
# # list_of_nodes.pop(net.get_node('ShakingFragilityB'))
# # list_of_nodes.pop(net.get_node('ShakingFragilityC'))
[ net.get_node_id(x) for x in list_of_nodes]

In [ ]:
# for node_handle in list_of_nodes:
#     # Get a single fragility node
#     child_node_name = net.get_node_id(node_handle)
    
#     print(child_node_name)

In [ ]:
child_node_names = ['LiquefactionFragility','LandslideFragility','TsunamiFragility']

In [ ]:
for child_node_name in child_node_names:
    print(child_node_name)
    # Get a single fragility node
#     child_node_name = net.get_node_id(node_handle)
    # Create a new node for the hazard
    new_node_name = re.sub('Fragility','',child_node_name)
    net.add_node(pysmile.NodeType.CPT,new_node_name)
    # Add the outcomes
#     if (new_node_name=='ShakingA')|(new_node_name=='ShakingB')|(new_node_name=='ShakingC'):
#         outcomes = ['low','medium','high']
    if new_node_name=='Landslide':
        outcomes = ['no','yes']
    elif new_node_name=='Tsunami':
        outcomes = ['zero','low','medium','high']
    elif new_node_name=='Liquefaction':
        outcomes = ['A','B','C','D']
    for outcome in outcomes:
        net.add_outcome(new_node_name,outcome)
    # Delete the premade states
    net.delete_outcome(new_node_name,'State0')
    net.delete_outcome(new_node_name,'State1')
    # Create connection between the hazard and fragility functions
    
    net.add_arc(new_node_name,child_node_name)
    print(new_node_name)
    print(child_node_name)
    # Set the values of the conditional probability table for the fragility node
    if new_node_name=='Landslide':
        cpt_value = \
            list(df_ds_landslide[df_ds_landslide.index==landslide_no].T[landslide_no])+\
            list(df_ds_landslide[df_ds_landslide.index==landslide_yes].T[landslide_yes])
    elif new_node_name=='Tsunami':
        cpt_value = \
            list(df_ds_tsunami[df_ds_tsunami.index==tsunami_zero].T[tsunami_zero])+\
            list(df_ds_tsunami[df_ds_tsunami.index==tsunami_low].T[tsunami_low])+\
            list(df_ds_tsunami[df_ds_tsunami.index==tsunami_medium].T[tsunami_medium])+\
            list(df_ds_tsunami[df_ds_tsunami.index==tsunami_high].T[tsunami_high])
    elif new_node_name=='Liquefaction':
        cpt_value = [1,0,0,0,1,0,0,0,1]
        
    net.set_node_definition(child_node_name,cpt_value)
    if new_node_name!='Liquefaction':
        net.set_node_definition(new_node_name,[1/len(outcomes)]*len(outcomes))

In [ ]:
sum([ls for ls in [[2,3,4],[2,3,4],[2,3,4]]],[])

In [ ]:

# Get a single fragility node
# child_node_name = net.get_node_id(node_handle)
# Create a new node for the hazard
new_node_name = 'Shaking'

net.add_node(pysmile.NodeType.CPT,new_node_name)

outcomes = ['low','medium','high']

for outcome in outcomes:
    net.add_outcome(new_node_name,outcome)
# Delete the premade states
net.delete_outcome(new_node_name,'State0')
net.delete_outcome(new_node_name,'State1')
# Create connection between the hazard and fragility functions

for node in ['ShakingFragilityA','ShakingFragilityB','ShakingFragilityC','ShakingFragilityD']:
    net.add_arc(new_node_name,node)

# Set the values of the conditional probability table for the fragility node
cpt_value = \
    list(df_ds_shaking_a[df_ds_shaking_a.index==earthquake_low].T[earthquake_low])+\
    list(df_ds_shaking_a[df_ds_shaking_a.index==earthquake_medium].T[earthquake_medium])+\
    list(df_ds_shaking_a[df_ds_shaking_a.index==earthquake_high].T[earthquake_high])
net.set_node_definition('A',cpt_value)

cpt_value = \
    list(df_ds_shaking_b[df_ds_shaking_b.index==earthquake_low].T[earthquake_low])+\
    list(df_ds_shaking_b[df_ds_shaking_b.index==earthquake_medium].T[earthquake_medium])+\
    list(df_ds_shaking_b[df_ds_shaking_b.index==earthquake_high].T[earthquake_high])
net.set_node_definition('B',cpt_value)

cpt_value = \
    list(df_ds_shaking_c[df_ds_shaking_c.index==earthquake_low].T[earthquake_low])+\
    list(df_ds_shaking_c[df_ds_shaking_c.index==earthquake_medium].T[earthquake_medium])+\
    list(df_ds_shaking_c[df_ds_shaking_c.index==earthquake_high].T[earthquake_high])
net.set_node_definition('C',cpt_value)

cpt_value = \
    list(df_ds_shaking_d[df_ds_shaking_d.index==earthquake_low].T[earthquake_low])+\
    list(df_ds_shaking_d[df_ds_shaking_d.index==earthquake_medium].T[earthquake_medium])+\
    list(df_ds_shaking_d[df_ds_shaking_d.index==earthquake_high].T[earthquake_high])
net.set_node_definition('D',cpt_value)


net.set_node_definition('Shaking',[1/3]*3)


In [ ]:
# Add connection between  shaking and liquefaction
net.add_arc('Shaking','Liquefaction')
net.set_node_definition('Liquefaction',[0.8,0.2,0,0.4,0.5,0.1,0.1,0.3,0.6])



In [ ]:
net.set_node_position('Shaking',400,50,50,50)
net.set_node_position('Liquefaction',700,50,50,50)
net.set_node_position('Tsunami',900,50,50,50)
net.set_node_position('Landslide',1100,50,50,50)

net.set_node_position('ShakingFragilityA',200,175,50,50)
net.set_node_position('ShakingFragilityB',400,175,50,50)
net.set_node_position('ShakingFragilityC',600,175,50,50)

net.set_node_position('ShakingFragility',400,300,50,50)
net.set_node_position('LiquefactionFragility',700,300,50,50)
net.set_node_position('TsunamiFragility',900,300,50,50)
net.set_node_position('LandslideFragility',1100,300,50,50)

net.set_node_position('CombinedFragility',800,425,50,50)

In [ ]:
for node in net.get_all_nodes():
    net.set_node_chart_enabled(node,1)


In [ ]:
net.write_file("test.xdsl")
# frag_model_dict['model'][0].update({
#     'net':net
# })


In [ ]:
# Add hazard evidence to dict
hazards_evidence_dict = {
    'Landslide':[1,0],
    'Tsunami':[0,1,0,0],
    'Shaking':[1,0,0]
}
frag_model_dict = BNModel().add_evidence_to_dict(frag_model_dict,hazards_evidence_dict)

# Set evidence and get beliefs
frag_model_dict = BNModel().update_evidence(frag_model_dict)


# Set Evidence

In [ ]:
mean_pga = 0.42482
sigma_pga = 2.1371


In [ ]:
frag_model_dict

# Looping over grid, calculating probs

In [ ]:
# Loop over each of the hazards, and get the relevant columns

In [ ]:
df_hazards = pd.read_csv('example_data.csv')
hazard_keys = ['_shaking_prob','_liq','_tsunami','_landslide']
hazard_node_names = ['Shaking','Liquefaction','Tsunami','Landslide']

for index,row in df_hazards.iterrows():
    
    # Create evidence dict, one hazard at a time
    hazards_evidence_dict = {}
    for key,node_name in zip(hazard_keys,hazard_node_names):
        hazard_cols = [x for x in df_hazards.columns if key in x]
            
        hazards_evidence_dict.update({
            node_name:list(row[hazard_cols])
        })
        asdf
    # Add hazard evidence to dict
    frag_model_dict = BNModel().add_evidence_to_dict(frag_model_dict,hazards_evidence_dict)
    
    # Set evidence and get beliefs
    frag_model_dict = BNModel().update_evidence(frag_model_dict)
    
    asdf

In [ ]:
net.get_node_value('Combined')

In [ ]:
dir(net)

In [ ]:
net.get_node_definition('Combined')

In [ ]:
net.get_outcome_ids('Combined')

In [ ]:
# Add hazard evidence to dict
frag_model_dict = BNModel().add_evidence_to_dict(frag_model_dict,hazards_evidence_dict)

# Set evidence and get beliefs
frag_model_dict = BNModel().update_evidence(frag_model_dict)

In [ ]:
hazards_evidence_dict

In [ ]:
net.get_all_node_ids()

In [ ]:
asdfasdf

# Get the cpt for deciding between the PGAs

In [ ]:
PGA1

## Create Definition of bins for each hazard

### Earthquake

# Data Prep

## Preprocessing

In [ ]:
df_hazards = pd.read_csv('example_data.csv')
df_hazards_subset = df_hazards.drop(['lon','lat'],axis=1)
hazards_data_dict = {col:np.array(content) for col,content in df_hazards_subset.iteritems()}

In [ ]:
df_hazards = df_hazards[1:]

In [ ]:
df

## Create Variable Dictionary

In [ ]:
#### Don't include spaces in bin names. if no discretisation, just leave out that key
hazards_model_dict = {
    'variables':{
        'meanpga_lo':{
            'label':'meanpga_lo',
            'discretisation':{
                'n_bins':3,
                'strategy':'kmeans',
                'bin_names':['Low','Mid','High']
            },
            'child_nodes':['Liq_Class','land_LP2500_max']
        },
        'Liq_Class':{
            'label':r'Liq_Class',
            'child_nodes':[],
            'bins':np.unique(df_hazards['Liq_Class'])
        },
        'land_LP2500_max':{
            'label':'land_LP2500_max',
            'discretisation':{
                'n_bins':3,
                'strategy':'kmeans',
                'bin_names':['Low','Mid','High']
            },
            'child_nodes':[]
        }
    },
    'training_frac':0.8,
    'bootstrap_reps':1
}

## Bootstrapping the data

In [ ]:
# Bootstrap the data, and add it to the model_dict
hazards_model_dict = BNModel().bootstrap_data(hazards_model_dict,hazards_data_dict,df_hazards)


In [ ]:
hazards_model_dict['variables'].keys()

## Discretise the data

In [ ]:
# Discretise the data
hazards_file_label = "hazards"

hazards_model_dict = BNModel().discretiser(hazards_model_dict,["Liq_Class"])

In [ ]:
# hazards_disc_fig = BNModel().plot_discretiser(hazards_model_dict,["Liq_Class"])

## Save the data

In [ ]:
BNModel().save_dataset(hazards_model_dict,hazards_file_label)

# Create BN Model

In [ ]:
# Create the BN
hazards_model_dict = BNModel().create_SM(hazards_model_dict,hazards_file_label)

## View Conditional Probability tables

In [ ]:
# Get conditional probs tables
BNModel().get_conditional_prob_table(hazards_model_dict,'land_LP2500_max')

In [ ]:
# # Get conditional probs tables
# df_CPT_MSL = BNModel().get_conditional_prob_table(ocean_model_dict,'TWL')

# df_CPT_MSL = df_CPT_MSL.loc[['VeryLow','Low','Mid','High','VeryHigh']]

# plt.pcolor(df_CPT_MSL)

# Set Evidence

In [ ]:
hazards_evidence_dict = {
    'land_LP2500_max':[0.05,0.1,0.01]
}


In [ ]:
hazards_model_dict = BNModel().add_evidence_to_dict(hazards_model_dict,hazards_evidence_dict)

# Update beliefs based on evidence

In [ ]:
# Set evidence and get beliefs
hazards_model_dict = BNModel().update_evidence(hazards_model_dict)

# BN Visualisation

In [ ]:

# Set up graph
graph_hazards = BNModel().create_BN_graph()

# Create nodes of the graph
graph_hazards,hazards_model_dict = BNModel().create_nodes(graph_hazards,hazards_model_dict,0)

# Create arcs between nodes
graph_hazards = BNModel().create_arcs(graph_hazards,hazards_model_dict)

# Save as dot file
graph_hazards.render(filename='graph_hazards',format='png')

# Plot the graph
graph_hazards
